In [9]:
import sys
sys.path.append('../') 

from config.reddit import REDDIT_CL_ID, REDDIT_CL_SECRET

import praw
import pandas as pd

In [10]:
user_agent = "scrapper 1.0"
# Credenciales de la aplicación
client_id = REDDIT_CL_ID
client_secret = REDDIT_CL_SECRET

# Crear una instancia de Reddit
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [11]:
df_publicaciones = pd.DataFrame(columns=['id', 'titulo', 'autor', 'fecha_submission'])
df_comentarios = pd.DataFrame(columns=['id_publicacion', 'usuario', 'fecha', 'comentario'])

In [17]:
for submission in reddit.subreddit('apple').hot(limit=50):
    submission_id = submission.id
    title = submission.title
    author = submission.author
    submission_date = submission.created_utc
    url = submission.url

    # Guardar los detalles de la publicación en el dataframe correspondiente
    df_nueva_publicacion = pd.DataFrame({'id': [submission_id],
                                        'titulo': [title],
                                        'autor': [author],
                                        'fecha_submission': [submission_date],
                                        'url': [url]})
    df_publicaciones = pd.concat([df_publicaciones, df_nueva_publicacion], ignore_index=True)

    # Reemplazar los objetos MoreComments con los comentarios reales
    submission.comments.replace_more(limit=None)
    comments = submission.comments.list()

    # Recorrer los comentarios de la publicación
    for comment in comments:
        comentario_submission_id = submission_id
        usuario = comment.author.name if comment.author else "N/A"
        fecha = comment.created_utc
        comentario = comment.body
        
        # Guardar los comentarios en el dataframe correspondiente
        df_nuevo_comentario = pd.DataFrame({'id_publicacion': [comentario_submission_id],
                                            'usuario': [usuario],
                                            'fecha': [fecha],
                                            'comentario': [comentario]})
        df_comentarios = pd.concat([df_comentarios, df_nuevo_comentario], ignore_index=True)


In [18]:
df_publicaciones

,id,titulo,autor,fecha_submission,url
0,1c4iexr,"Daily Advice Thread - April 15, 2024",AutoModerator,1.713175e+09,https://www.reddit.com/r/apple/comments/1c4iex...
1,1c4r3hf,Apple Further Explains Why Game Boy Emulator i...,digidude23,1.713200e+09,https://www.macrumors.com/2024/04/15/apple-fur...
2,1c4fwk0,Apple Removes Game Boy Emulator iGBA From App ...,Salanderfan14,1.713165e+09,https://www.macrumors.com/2024/04/15/apple-rem...
3,1c4j62z,iPhone 16 Pro Camera May Be Less Prone to Lens...,favicondotico,1.713178e+09,https://www.macrumors.com/2024/04/15/iphone-16...
4,1c49j8g,Apple's First AI Features in iOS 18 Reportedly...,Drtysouth205,1.713143e+09,https://www.macrumors.com/2024/04/14/apples-fi...
...,...,...,...,...,...
147,1bzugpx,CT scans of the Vision Pro show how its design...,jonbruner,1.712675e+09,https://www.lumafield.com/article/apple-vision...
148,1bzu226,Apple teaching an AI system to use apps; maybe...,favicondotico,1.712674e+09,https://9to5mac.com/2024/04/09/ferret-ui-advan...
149,1bzo1rw,Apple pivots to housing for Indian factory wor...,utkarsh_aryan,1.712656e+09,https://www.business-standard.com/companies/ne...
150,1bzuqx6,Beeper acquired by Automattic (Wordpress and W...,somuchlan,1.712676e+09,https://blog.beeper.com/2024/04/09/beeper-is-j...


In [19]:
df_comentarios

,id_publicacion,usuario,fecha,comentario
0,1c4iexr,DrJiz,1.713196e+09,Are there any updates on the possible announce...
1,1c4iexr,clarksworth,1.713180e+09,"UK Apple store - if using Barclays finance, is..."
2,1c4iexr,JonathanMa021703,1.713184e+09,Is the app store not working for anyone else? ...
3,1c4iexr,The_Sesquipedalian,1.713206e+09,"No new news as of recently, but the latest rum..."
4,1c4r3hf,digidude23,1.713200e+09,">Notably, Apple confirmed to us that emulators..."
...,...,...,...,...
22852,1bzfck4,CyberBot129,1.712669e+09,Elon Musk’s entire empire is built off of the ...
22853,1bzfck4,bran_the_man93,1.712629e+09,I guess my point is that these things are so r...
22854,1bzfck4,TJPrime_,1.712630e+09,Depends where you look on Reddit. r/Apple is a...
22855,1bzfck4,bran_the_man93,1.712630e+09,"Meh, I think even getting a serious audience w..."


In [20]:
df_publicaciones.to_csv("../data/reddit_publications.csv", index=False)
df_comentarios.to_csv("../data/reddit_coments.csv", index=False)